In [347]:
import numpy as np
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#conda install Basemap
#from mpl_toolkits.basemap import Basemap
from itertools import combinations
pd.set_option('display.max_columns', 20000)
import datetime

In [344]:
eq = pd.read_csv('data/03_equipment.csv', sep = ",")

In [305]:
data = pd.read_csv('data/02_Data_test.csv', sep = ';')

In [306]:
positive_labels = pd.read_csv('data/01_Facts.csv', sep = ";", header = None)

In [307]:
data.head(1)

,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,5029,40798,158529599791,3.531111e+14,0,1369252800974,37.9305,55.7815,2101,OUTDOOR,220.0,280.0


"Msisdn" column is how many milliseconds passed from 1970-01-01. The next function convert this column in ordinary date-time format

In [308]:
data['tstamp'] = data['tstamp'].apply(lambda x: datetime.datetime(1970, 1, 1, 0, 0) + datetime.timedelta(milliseconds = x))

In [309]:
data.head(1)

,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,5029,40798,158529599791,3.531111e+14,0,2013-05-22 20:00:00.974,37.9305,55.7815,2101,OUTDOOR,220.0,280.0


In [310]:
print("How many observations in our data set? -  {}".format(len(data)))
print("How many unique numbers (msisdn) in our data set? - {}".format(data['msisdn'].nunique()))

How many observations in our data set? -  174123
How many unique numbers (msisdn) in our data set? - 2361


In [311]:
list_of_positive_labels = []
for i in range(len(positive_labels)):
    for j in range(2):
        list_of_positive_labels.append(positive_labels.loc[i, j])

In [312]:
data_small = pd.DataFrame(columns = data.columns)
for i in list(data['msisdn'].unique()):
    if i in list_of_positive_labels:
        data_small = data_small.append(data[data['msisdn'] == i])
data_small = data_small.reset_index(drop = True)

In [313]:
data_small.head()

,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,7798,3866,158528850493,NaN,0,2013-05-22 20:00:34.051,37.6735,55.7363,403,OUTDOOR,255.0,315.0
1,7761,15621,158528850493,NaN,0,2013-05-22 20:05:32.222,37.6665,55.7385,337,OUTDOOR,170.0,230.0
2,7798,3866,158528850493,1.300201e+13,3,2013-05-22 20:05:32.299,37.6735,55.7363,403,OUTDOOR,255.0,315.0
3,7798,3866,158528850493,NaN,0,2013-05-22 20:05:52.222,37.6735,55.7363,403,OUTDOOR,255.0,315.0
4,7761,15621,158528850493,1.300201e+13,3,2013-05-22 23:10:34.644,37.6665,55.7385,337,OUTDOOR,170.0,230.0


In [314]:
len(data_small)

8213

Since cid - identificator of base station unique in terms of lac the folowing cell creates columns 'lac_cid' as a combination of cid and lac and this gives as unique location for each station

In [315]:
data_small['lac_cid'] = data_small['lac'].astype(str)  + data_small['cid'].astype(str)

Grouping by msisdn. The first column of result dataframe msisdn itself, the second column - list of all locations where this number was registered

In [316]:
group = data_small.groupby(['msisdn'], as_index=False).agg({'lac_cid':(lambda x: list(x))}).rename(columns={'msisdn':'number', 'lac_cid':'set_'})

In [317]:
len(group)

241

Creating all posible combinations of msisdn

In [318]:
df = pd.DataFrame(columns = ['number_1', 'set_1', 'number_2', 'set_2'])

for index in list(combinations(group.index,2)):
    
    df0 = pd.DataFrame(group.loc[index[0], :]).T.reset_index(drop=True)
    df1 = pd.DataFrame(group.loc[index[1], :]).T.reset_index(drop=True)
    df_temp = pd.concat([df0, df1], axis = 1)
    df_temp.columns = ['number_1', 'set_1', 'number_2', 'set_2']
    df = df.append(df_temp)
df = df.set_index(pd.Index(range(len(df))))

In [319]:
df.head()

,number_1,set_1,number_2,set_2
0,158500411793,"[502024424, 50205834, 502041356, 502049184, 50...",158500437133,"[775331576, 775334177, 775331576, 775334177, 7..."
1,158500411793,"[502024424, 50205834, 502041356, 502049184, 50...",158500611579,"[77125754, 77125754, 771220486, 771220486]"
2,158500411793,"[502024424, 50205834, 502041356, 502049184, 50...",158500670731,"[508648686, 502045597, 502048679, 508648675]"
3,158500411793,"[502024424, 50205834, 502041356, 502049184, 50...",158500730293,"[776530546, 775530067, 778432289, 778432289, 7..."
4,158500411793,"[502024424, 50205834, 502041356, 502049184, 50...",158500730487,"[77658275, 7794700, 775522653, 775522653, 7755..."


In [320]:
len(df)

28920

In [324]:
for i in range(28920):
    if df.iloc[i, 0] == df.iloc[i, 2]:
        df = df.drop(i)

The first assumption is that if user holds 2 numbers the locations rigistered by base station of exactly these numbers intersect

Let's create the column 'inter' which contains the information of how much intersections for each row (pair of numbers) we have over length of the shotest list among two given (list of locations of two candidates). It's important because if the first list consists of 20 elements, and the second only of 3 but all these 3 fall into those 20, we will have only 3 intersections. By actually taking into account the fact that the  registered locations of the second number 100% the same as locations of the first number, gives us the assumption that they might have the same user (may he brings the second number rarely, but he always goes in the same places).

In [325]:
def intersect(x):
    list1 = x['set_1']
    list2 = x['set_2']
    
    res = []
    list_1 = list1.copy()
    for i in list_1:
        if i in list2:
            res.append(i)
            list1.remove(i)
            list2.remove(i)
            
    min_len = min(len(list1), len(list2))
    
    
    return len(res)/(min_len+1)

In [327]:
df['inter'] = df.apply(intersect, axis =1)

In [ ]:
The following finction returns the dictionary of possible numbers which belong to the same person. It chooses candidates by selecting maximum intersection between them

In [336]:
def find_pair(df):
    res = {}
    for number_1 in df['number_1'].unique():
        value = max(df[df['number_1'] == number_1]['inter'])
        if value > 0.1:
            number_2 = df[(df['number_1'] == number_1) &(df['inter'] == value)]['number_2'].iloc[0]
            res[number_1] = number_2
    return res

In [337]:
res = find_pair(df)

In [338]:
res

{158500411793: 158529027621,
 158500437133: 158506811071,
 158500730293: 158533303379,
 158500941403: 158516497707,
 158501037171: 158522529493,
 158501551109: 158505222935,
 158501557263: 158506092285,
 158502336809: 158532486825,
 158502683611: 158520842209,
 158502911917: 158516597249,
 158503439635: 158525444655,
 158503655953: 158527183389,
 158503697375: 158507587401,
 158503800973: 158508415291,
 158503845079: 158517547861,
 158503851673: 158528849161,
 158504189539: 158529872369,
 158504562579: 158516997599,
 158505222935: 158521773577,
 158505407647: 158526876459,
 158505653139: 158505653213,
 158507025965: 158509296887,
 158507158161: 158529353319,
 158507183223: 158523324811,
 158507531557: 158528710497,
 158507606357: 158522532707,
 158507689477: 158526742489,
 158508364923: 158528850493,
 158508415291: 158532486825,
 158508437897: 158527185851,
 158508570305: 158528861475,
 158508828513: 158521057825,
 158509296887: 158524011355,
 158510090027: 158520030347,
 158510912201:

In [339]:
len(res)

116

In [153]:
df.head()

,number_1,set_1,number_2,set_2,inter
0,158500411793,"[50205834, 502049184, 502049184, 50209186, 502...",158500437133,"[775334176, 775334177, 775334176, 775334177, 7...",0.0
1,158500411793,"[50205834, 502049184, 502049184, 50209186, 502...",158500611579,"[77125754, 77125754, 771220486, 771220486, 771...",0.0
2,158500411793,"[50205834, 502049184, 502049184, 50209186, 502...",158500670731,"[508648686, 502045597, 502048679, 508648675, 5...",0.0
3,158500411793,"[50205834, 502049184, 502049184, 50209186, 502...",158500730293,"[775530067, 778432289, 778432289, 776537843, 7...",0.0
4,158500411793,"[50205834, 502049184, 502049184, 50209186, 502...",158500730487,"[775522653, 775522653, 775512588, 775522653, 7...",0.0


Let see if all our observations belong to the same country(city), by depicting the coordinates of unique groups of base stations

#Choose unique group of base stations coordinates
a = []
for lac in list(data['lac'].unique()):
    a.append((data[data['lac'] == lac].head(1)[['long', 'lat']].iloc[0, 0], data[data['lac'] == lac].head(1)[['long', 'lat']].iloc[0, 1]))

#Depict them on map
fig = plt.figure(figsize=(8, 10), edgecolor='w')
m = Basemap(projection='cyl', resolution=None,
            llcrnrlat=40, urcrnrlat=70,
            llcrnrlon=0, urcrnrlon=80, )
m.etopo(scale=1.5, alpha=0.5)

# Map (long, lat) to (x, y) for plotting
for i in a:
    x, y = m(i[0], i[1])
    plt.plot(x, y, 'ok', markersize=1)
#plt.text(x, y, ' Seattle', fontsize=12);

All right, seems that they all belong to one country, one city - must be Moscow